In [ ]:
# Generate a API access point

import requests
import json

class LLMClient:
    def __init__(self, base_url: str = "http://10.252.38.241:8000"):
        self.base_url = base_url
        self.session_token = None
        self.headers = {"Content-Type": "application/json"}

    def login(self, username: str, password: str) -> bool:
        """Login to the LLM API and return a session token."""
        response = requests.post(
            f"{self.base_url}/api/auth/login",
            headers = self.headers,
            json={"username": username, "password": password}
        )
        if response.status_code == 200:
            data = response.json()
            self.session_token = data["session_token"]
            self.headers["Authorization"] = f"Bearer {self.session_token}"
            print(f"Login successful. Session token: {self.session_token}")
            return True
        else:
            print(f"Login failed. Status code: {response.status_code}") 
            return False
    
    def chat(self, message, session_id=None, chat_type="normal", json_path=None) -> str:
        """Send a message to the LLM API and return the response."""
        endpoint_map = {
            "normal": "/api/chat",
            "web_search": "/api/chat/web-search",
            "rag": "/api/chat/rag",
            "json": "/api/chat/with-json"
        }
        endpoint = endpoint_map.get(chat_type, "/api/chat")
        payload = {"message": message}

        if chat_type == "json":
            with open(json_path, "r") as fp:
                json_contents = json.load(fp)

            inline_payload = {
                "message": message,
                "json_data": json_contents,
                "session_id": session_id,
                "temperature": 0.7,  # Optional overrides
            }

            response = requests.post(
                f"{self.base_url}/api/chat/with-json",
                headers=self.headers,
                json=inline_payload,
                timeout=600,
            )

        if session_id:
            payload["session_id"] = session_id
            
        response = requests.post(
            f"{self.base_url}{endpoint}",
            headers = self.headers,
            json = payload
        )
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error: {response.status_code}")
        return None
        
    def list_conversations(self) -> list:
        """List all conversations."""
        response = requests.get(
            f"{self.base_url}/api/chat/conversations",
            headers = self.headers
        )
        return response.json()

    def create_conversation(self, name: str = None) -> str:
        """Create a new conversation."""
        response = requests.post(
            f"{self.base_url}/api/conversations",
            headers = self.headers,
        )
        return response.json().get("session_id")

    def get_conversation(self, session_id: str):
        """Get a conversation by session ID."""
        response = requests.get(
            f"{self.base_url}/api/conversations/{session_id}/history",
            headers = self.headers
        )
        return response.json()
    
    def configure_model(self, model_name, context_size, temperature):
        """Configure the model for a conversation."""
        response = requests.post(
            f"{self.base_url}/api/admin/users",
            headers = self.headers,
            json = {
                "model": model_name,
                "temperature": temperature,
                "context_size": context_size,
                "top_p": 0.9,
                "top_k": 50,
                "num_ctx": context_size,
                "timeout": 30000
            }
        )
        return response.json()
    
    def create_user(self, username, password, email, display_name):
        """Create a new user."""
        response = requests.post(
            f"{self.base_url}/api/admin/users",
            headers = self.headers,
            json = {
                "username": username,
                "password": password,
                "email": email,
                "display_name": display_name,
                "role": "user"
            }
            )
        return response.json()

In [ ]:
client = LLMClient()
client.login("admin", "administrator")
client.create_user("user", "password", "user@example.com", "User")


In [ ]:
client.login("user", "password")
result = client.configure_model("gpt-oss:20b", 8192, 0.5)
print(result)

In [ ]:
result = client.chat("Hello, how are you?")
if result:
    session_id = result["session_id"]
    print(result['response'])
    print(result['session_id'])

In [ ]:
print('Chat continues...')
print('\n')
print('\n')
print('\n')

result = client.chat("What is the capital of France?", session_id=session_id)

print(result['response'])

In [ ]:
print('Performing web search...')
print('\n')
print('\n')
print('\n')

result = client.chat("What is the capital of France?", session_id=session_id, chat_type="web_search")
print(result['response'])

print(result['session_id'])
print(result['response'])
print(f'WEB SEARCH RESULTS USED: {result.get('search_results_used', 0)}')


In [ ]:
print('Show conversation history...')
print('\n')
print('\n')
print('\n')

history = client.get_conversation(session_id)
print(history)

In [ ]:
print('Work with JSON...')
print('\n')
print('\n')
print('\n')


result = client.chat("What is the average warpage?", json_path="./warpage/20250925_stats.json", chat_type="json")